In [ ]:
import re
import win32com.client

METHOD_HDR = re.compile(r"(?m)^\s*//\s*METHOD\s+(?P<name>\w+)\b.*$")

def split_blob_into_body_and_methods(st_blob: str):
    st_blob = (st_blob or "").replace("\r\n", "\n").replace("\r", "\n")

    headers = list(METHOD_HDR.finditer(st_blob))
    if not headers:
        return st_blob.strip(), {}

    body = st_blob[:headers[0].start()].strip()
    methods = {}
    for i, h in enumerate(headers):
        name = h.group("name")
        start = h.end()
        end = headers[i + 1].start() if i + 1 < len(headers) else len(st_blob)
        methods[name] = st_blob[start:end].strip()
    return body, methods


In [ ]:
import win32com.client
import pythoncom

def create_dte():
    prog_ids = [
        "TcXaeShell.DTE.17.0", "VisualStudio.DTE.17.0",
        "TcXaeShell.DTE.16.0", "VisualStudio.DTE.16.0",
        "TcXaeShell.DTE.15.0", "VisualStudio.DTE.15.0",
        "VisualStudio.DTE.14.0", "VisualStudio.DTE.12.0", "VisualStudio.DTE.10.0",
    ]
    last_err = None
    for pid in prog_ids:
        try:
            dte = win32com.client.Dispatch(pid)
            dte.SuppressUI = True
            try:
                settings = dte.GetObject("TcAutomationSettings")
                settings.SilentMode = True
            except Exception:
                pass
            return dte
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Kein passender DTE ProgID gefunden. Letzter Fehler: {last_err}")

def open_solution_get_sysman(solution_path: str):
    dte = create_dte()
    dte.Solution.Open(solution_path)

    tc_project = None
    for i in range(1, dte.Solution.Projects.Count + 1):
        p = dte.Solution.Projects.Item(i)
        if str(p.FullName).lower().endswith(".tsproj"):
            tc_project = p
            break

    if tc_project is None:
        raise RuntimeError("Kein TwinCAT .tsproj in der Solution gefunden.")

    sysman = tc_project.Object  # ITcSysManager
    return dte, sysman


In [ ]:
def enum_children(tree_item):
    # wie in plcopen_withHW.py: erst iterator, dann ChildCount/Child(i) :contentReference[oaicite:2]{index=2}
    children = []
    try:
        for child in tree_item:
            children.append(child)
    except Exception:
        cnt = int(tree_item.ChildCount)
        for i in range(1, cnt + 1):
            children.append(tree_item.Child(i))
    return children

def find_first_plc_nested_project(sysman):
    root_plc = sysman.LookupTreeItem("TIPC")
    for child in enum_children(root_plc):
        try:
            return child.NestedProject  # klappt nur bei PLC-Projekt-Knoten :contentReference[oaicite:3]{index=3}
        except pythoncom.com_error:
            continue
    raise RuntimeError("Kein PLC-Projekt unter TIPC gefunden (NestedProject klappt nirgends).")

def get_pous_tree_path(sysman):
    plc = find_first_plc_nested_project(sysman)
    pous_path = f"{plc.PathName}^POUs"
    pous_item = sysman.LookupTreeItem(pous_path)
    return pous_item, pous_path


In [ ]:
def upsert_job_fb_and_save(
    solution_path: str,
    fb_name: str,
    fb_decl: str,
    st_blob: str,
    pous_tree_path: str = None,      # optional; wenn None -> dynamisch
    method_return_type: str = "",
    method_access: str = "PUBLIC",
):
    dte, sysman = open_solution_get_sysman(solution_path)

    try:
        if pous_tree_path is None:
            pous_item, pous_tree_path = get_pous_tree_path(sysman)
        else:
            pous_item = sysman.LookupTreeItem(pous_tree_path)

        # FB holen/erstellen (604 = Function Block, 6 = ST)
        fb_path = f"{pous_tree_path}^{fb_name}"
        try:
            fb_item = sysman.LookupTreeItem(fb_path)
        except Exception:
            fb_item = pous_item.CreateChild(fb_name, 604, "", 6)

        fb_body_impl, methods = split_blob_into_body_and_methods(st_blob)

        fb_item.DeclarationText = fb_decl
        fb_item.ImplementationText = fb_body_impl

        # Methoden erstellen/aktualisieren (609 = Method)
        for mname, impl in methods.items():
            m_path = f"{fb_path}^{mname}"
            try:
                m_item = sysman.LookupTreeItem(m_path)
            except Exception:
                # Einfacher Default: ST-Methode
                m_item = fb_item.CreateChild(mname, 609, "", 6)

            m_item.ImplementationText = impl

        dte.ExecuteCommand("File.SaveAll")

    finally:
        dte.Quit()


In [ ]:
SOLUTION = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln"
FB_NAME = "VSG_AS_VerticalMoveEncoder"

ST_BLOB = """// POU VSG_AS_VerticalMoveEncoder body
HRL_RGB_VerticalMoveWithEncoder(
    MethodCall := MethodCall_VerticalMove,
    EmergencyStopSignal := ESS
);
DO_MovingVerticalVSG_01 := HRL_RGB_VerticalMoveWithEncoder.DigitalOutputControl_01;

// METHOD Abort of VSG_AS_VerticalMoveEncoder
callCounterAbort := callCounterAbort + 1;

// METHOD CheckState of VSG_AS_VerticalMoveEncoder
callCounterCheckState := callCounterCheckState + 1;

// METHOD Start of VSG_AS_VerticalMoveEncoder
callCounterStart := callCounterStart + 1;
"""

FB_DECL = f"""FUNCTION_BLOCK {FB_NAME}
VAR
    (* TODO: Variablen *)
END_VAR
"""

upsert_job_fb_and_save(
    solution_path=SOLUTION,
    fb_name=FB_NAME,
    fb_decl=FB_DECL,
    st_blob=ST_BLOB,
    pous_tree_path=None,  # <- dynamisch!
)
